In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [8]:
import unicodedata, re
from tqdm.notebook import tqdm

In [9]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(insta_url)
time.sleep(1)

In [10]:
with open('insta.txt') as f:
    password = f.read()

### 1. 로그인하기

In [11]:
email = 'yyewon99@gmail.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [12]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)

In [13]:
input_pwd.submit()
time.sleep(1)

In [14]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [15]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주맛집으로 검색

In [16]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%A7%9B%EC%A7%91'

In [17]:
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글

In [18]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [19]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [20]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'#제주맛집산도롱맨도롱----------🍜갈비국수 대표맛집 🍜📌 산도롱의 대표메뉴 홍갈비국수와 백갈비국수 💕📌 진하게 우려낸 사골육수로 더욱 담백하게 ~!!!📌 제주의 신선한 문어가 풍덩~! 톳문어전 🥰..#성산일출봉맛집.대기시간이 비교적 없는 아침타임!성산일출봉을 다녀온후 아침드시기 좋습니다😙.산도롱맨도롱영업시간:07:00~17:00✔매주 화요일은 휴무입니다✔문의 : 064-782-5105.#제주도고기국수#광치기해변맛집#제주돔베고기#제주도점심#성산일출봉근처#성산일출봉아침식사'

In [21]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#제주맛집산도롱맨도롱----------🍜갈비국수',
 '#성산일출봉맛집.대기시간이',
 '#제주도고기국수',
 '#광치기해변맛집',
 '#제주돔베고기',
 '#제주도점심',
 '#성산일출봉근처',
 '#성산일출봉아침식사']

In [22]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-08'

In [24]:
# 4) 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [25]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

'산도롱맨도롱'

In [28]:
row = [content, date, like, place, tags]

### 4. 다음 게시글로 이동

In [26]:
# 첫 번째 게시글로 부터 두 번째 글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div').click()
time.sleep(2)

In [27]:
# 두 번째 게시글부터 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button/div').click()
time.sleep(2)

In [29]:
# 5)을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 5. 반복문으로 데이터 가져오기

In [30]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [31]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [33]:
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button/div').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div').click()
        time.sleep(3)

  0%|          | 0/200 [00:00<?, ?it/s]

In [34]:
driver.close()

### 6. 데이터프레임으로 만들고 중복 제거

In [35]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,도너츄 먹고싶다 🤤🍩,2021-12-07,121,카페 노티드 제주 애월 - Cafe Knotted Jeju Aewol,[]
1,#그루브제주 @groov.jeju 🗺 제주시 한림읍 한림로 333 (협재)⏰ 10:...,2021-12-07,mpany_님 여러 명이 좋아합니,GROOV JEJU,[#그루브제주]
2,#그랜드조선#제주#힐링#힐링은휴식#먹스타그램#중문맛집#갈치조림#갈치튀김#갈치구이#해...,2021-12-08,34,그랜드조선 제주,"[#그랜드조선, #제주, #힐링, #힐링은휴식, #먹스타그램, #중문맛집, #갈치조..."
3,애월 로드129성게크림파스타흑돼지 멘치카츠흑돼지 치즈돈까스흑돼지 고구마치즈돈까스흑돼...,2021-12-08,저 좋아요를 눌러보세,,"[#애월맛집, #제주맛집, #애월읍맛집, #애월항맛집, #제주애월점심, #애월맛집추..."
4,애월 로드129성게크림파스타흑돼지 멘치카츠흑돼지 치즈돈까스흑돼지 고구마치즈돈까스흑돼...,2021-12-08,저 좋아요를 눌러보세,,"[#애월맛집, #제주맛집, #애월읍맛집, #애월항맛집, #제주애월점심, #애월맛집추..."


In [36]:
df.drop_duplicates(subset='content', keep='first', inplace=True)
df.shape

(152, 5)

In [38]:
df.place.value_counts().to_frame()

,place
,73
환상의 섬 Jeju Island,4
Jeju-do,3
문쏘,3
브라보비치-제주,2
...,...
깡촌흑돼지,1
Road129,1
GROOV JEJU,1
쌩텀펜션,1


In [39]:
import requests
from urllib.parse import quote
import pandas as pd

In [42]:
with open('roadapikey.txt') as file:
    road_key = file.read()

In [43]:
quote('그랜드조선 제주')

'%EA%B7%B8%EB%9E%9C%EB%93%9C%EC%A1%B0%EC%84%A0%20%EC%A0%9C%EC%A3%BC'

In [44]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do' 
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("그랜드조선 제주")}&resultType=json'
url = f"{road_url}?{params1}&{params2}"

In [45]:
result = requests.get(url).json()
result

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '0',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': []}}

In [46]:
result['results']['juso'][0]['roadAddr']

IndexError: list index out of range